In [2]:
#### 0.1.1 - 2025 - 06 - 01
#### Dr. Marco Antonio Aceves
#### rev en Jupyter Notebook
#### Código como ejemplo como parte del libro:
#### de 0 a 100 en Inteligencia Artificial
#### 5_Q-Stacking

In [7]:
#Importar librerías
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score

In [8]:
# --- 1. Generar datos sintéticos ---
# Vamos a crear un conjunto de datos con 1000 muestras, 20 características (columnas)
# de las cuales 10 son informativas, y 2 clases.
X, y = make_classification(n_samples=1000, n_features=20, n_informative=10, n_redundant=10, random_state=42)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"Dimensiones de X_train: {X_train.shape}")
print(f"Dimensiones de X_test: {X_test.shape}")
print(f"Dimensiones de y_train: {y_train.shape}")
print(f"Dimensiones de y_test: {y_test.shape}\n")



Dimensiones de X_train: (700, 20)
Dimensiones de X_test: (300, 20)
Dimensiones de y_train: (700,)
Dimensiones de y_test: (300,)



In [9]:
# Definir los clasificadores base (los "expertos") ---
# Los clasificadores base deben ser una lista de tuplas,
# donde cada tupla es (nombre_del_estimador, estimador_instanciado).
base_classifiers = [
    ('logreg', LogisticRegression(random_state=42)),
    ('dtree', DecisionTreeClassifier(random_state=42)),
    ('knn', KNeighborsClassifier() ) # KNN no necesita random_state aquí
]

# Definir el meta-clasificador (el "combinador inteligente") ---
# Este es el modelo que aprenderá a combinar las predicciones de los base_classifiers.
meta_classifier = LogisticRegression(random_state=42)



In [10]:
# Crear el modelo Stacking ---
# StackingClassifier es la clase de scikit-learn para implementar stacking.
# estimators: La lista de clasificadores base.
# final_estimator: El meta-clasificador.
# cv: Número de folds para la validación cruzada interna (importante para evitar fugas de datos).
stacked_model = StackingClassifier(
    estimators=base_classifiers,
    final_estimator=meta_classifier,
    cv=5, # Usa 5-fold cross-validation para generar las predicciones para el meta-clasificador
    passthrough=False # Si es True, pasa las características originales al meta-clasificador también.
                      # Lo mantenemos en False para un ejemplo más puro de meta-aprendizaje de predicciones.
)

# Entrenar el modelo Stacking ---
print("Entrenando el modelo Stacking...")
stacked_model.fit(X_train, y_train)
print("Entrenamiento completado.\n")

# Evaluar el rendimiento del modelo Stacking ---
y_pred_stacked = stacked_model.predict(X_test)
accuracy_stacked = accuracy_score(y_test, y_pred_stacked)
print(f"Precisión del modelo Stacking: {accuracy_stacked:.4f}\n")

# (Opcional) Comparar con el rendimiento de los clasificadores base individuales ---
print("Rendimiento de los clasificadores base individuales:")
for name, model in base_classifiers:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"  Precisión de {name}: {accuracy:.4f}")

Entrenando el modelo Stacking...
Entrenamiento completado.

Precisión del modelo Stacking: 0.9333

Rendimiento de los clasificadores base individuales:
  Precisión de logreg: 0.8567
  Precisión de dtree: 0.8267
  Precisión de knn: 0.9500


/Users/marcoaceves/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
